In [4]:
import pandas
import csv
import os
from tqdm import tqdm

from pytest import skip


df_user_info = pandas.read_csv('user_watch_list_more_than_20.csv')
df_anime = pandas.read_csv('new_anime.csv')
df_anime_test = pandas.read_csv('anime_test.csv')

In [12]:
#create list of anime in test set 
test_anime = []
for i in range(len(df_anime_test)):
    test_anime.append(df_anime_test.loc[i]['Id'].replace('\'',''))


In [ ]:
#creates bag of words with all features
def create_soup(x):
    result = x['Anime Title']

    if x['Studio'] != 'n/a':
        result += ' ' + x['Studio']
    if x['Source Material'] != 'n/a':
        result += ' ' + x['Source Material']
    if x['Genre'] != 'n/a':
        result += ' ' + x['Genre']
    if x['VAs'] != 'n/a':
        result += ' ' + x['VAs']

    punc = '[],\'\"'
    
    for ele in result:
        if ele in punc:
            result = result.replace(ele, "")

    result
    return result

In [ ]:
#creates bag of words with reduced features
def create_soup_reduced(x):
    result = x['Anime Title']

    if x['Studio'] != 'n/a':
        result += ' ' + x['Studio']
    # if x['Source Material'] != 'n/a':
    #     result += ' ' + x['Source Material']
    if x['Genre'] != 'n/a':
        result += ' ' + x['Genre']
    # if x['VAs'] != 'n/a':
    #     result += ' ' + x['VAs']

    punc = '[],\'\"'
    
    for ele in result:
        if ele in punc:
            result = result.replace(ele, "")

    result
    return result

In [ ]:
#creates bag of words with reduced features
def create_soup_genres(x):
    result = x['Anime Title']

    # if x['Studio'] != 'n/a':
    #     result += ' ' + x['Studio']
    # if x['Source Material'] != 'n/a':
    #     result += ' ' + x['Source Material']
    if x['Genre'] != 'n/a':
        result += ' ' + x['Genre']
    # if x['VAs'] != 'n/a':
    #     result += ' ' + x['VAs']

    punc = '[],\'\"'
    
    for ele in result:
        if ele in punc:
            result = result.replace(ele, "")

    result
    return result

In [ ]:
#creates bag of words as a set
def create_soup_set(x):
    result = x['Anime Title']

    if x['Studio'] != 'n/a':
        result += ' ' + x['Studio']
    if x['Source Material'] != 'n/a':
        result += ' ' + x['Source Material']
    if x['Genre'] != 'n/a':
        result += ' ' + x['Genre']
    if x['VAs'] != 'n/a':
        result += ' ' + x['VAs']

    punc = '[],\'\"'
    
    for ele in result:
        if ele in punc:
            result = result.replace(ele, "")

    result = set(result.split())
    return result

In [ ]:
#read each user's watch list and creates a bag of words of all anime they watched
result = []
length = len(df_user_info)

# text = 34156
# print(text)
# print(create_soup(df_anime.loc[df_anime['Id'] == text.index[0]]))

for j in tqdm(range(length)):
    try:
        user = df_user_info.loc[j]['user']
        watched = (df_user_info.loc[j]['watched list']).split()
        bag = ''
        for k in range(len(watched)):
            if watched[k] not in test_anime:
                try:
                    temp = '\'' + watched[k] + '\''
                    index = df_anime.loc[df_anime['Id'] == temp].index[0]
                    bag += ' ' + create_soup_genres(df_anime.loc[index])
                except:
                    pass
    except:
        pass
    result.append([user,bag])

In [ ]:
print(result[2])

In [ ]:
outputdf = pandas.DataFrame(result, columns=['user','bag of words'])
outputdf.to_csv('users_bag_of_words_genres.csv', index=False)

In [ ]:
#read each user's watch list and creates a bag of words of all anime they watched
def make_bag(func):
    result = []
    length = len(df_user_info)

    # text = 34156
    # print(text)
    # print(create_soup(df_anime.loc[df_anime['Id'] == text.index[0]]))

    for j in range(1):
        try:
            user = df_user_info.loc[j]['user']
            watched = (df_user_info.loc[j]['watched list']).split()
            bag = ''
            for k in range(len(watched)):
                if watched[k] not in test_anime:
                    try:
                        temp = '\'' + watched[k] + '\''
                        index = df_anime.loc[df_anime['Id'] == temp].index[0]
                        bag += ' ' + func(df_anime.loc[index])
                        print(bag)
                    except:
                        pass
        except:
            pass
        result.append([user,bag])
    return result

In [ ]:
#create empty dataframe
header = ['anime']
for i in range(len(df_user_info)):
    header.append(df_user_info.loc[i]['user'])

result = []
for i in range(len(df_anime)):
    row = [df_anime.loc[i]['Id'].replace('\'','')]
    for j in range(len(df_user_info)):
        row.append(0)
    result.append(row)



In [ ]:
print(result[0])

In [ ]:
outputdf = pandas.DataFrame(result, columns=header)
outputdf.to_csv('anime_matrix.csv', index=False)

In [ ]:
df_matrix = pandas.read_csv('anime_matrix.csv')

print(df_matrix.head())

In [ ]:
#fills in the true values for each anime based on user's watch list
u = []
for i in range(len(df_user_info)):
    u.append(df_user_info.loc[i]['user'])
    
for j in tqdm(range(len(u))):
    try:
        w = df_user_info.loc[j]["watched list"].split()
        for i in range(len(df_anime)):
            # print(df_matrix.at[i,'ilampan'])
            if df_anime.loc[i]['Id'].replace('\'','') in w:
                df_matrix.at[i,u[j]] = 1
    except:
        pass

# print(df_matrix.at[0,'ilampan'])



In [ ]:
temp = df_user_info.loc[0]["watched list"].split()
print(temp[0])

In [ ]:
anime = []
for i in range(len(df_anime)):
    anime.append(df_anime.loc[i]['Id'])

users_watched = []
header = ['anime']
for i in range(len(df_user_info)):
    header.append(df_user_info.loc[i]['user'])
    try:
        users_watched.append(df_user_info.loc[i]["watched list"].split())
    except:
        users_watched.append([])    #append empty list if the user has no watch list


In [ ]:
#creates anime watched true values
result = []
for i in tqdm(range(len(anime))):
    id = anime[i].replace('\'','')
    temp = [id]
    for j in range(len(users_watched)):
        if id in users_watched[j]:
            temp.append(1)
        else:
            temp.append(0)
    result.append(temp)

In [ ]:
print((header[1]))

In [ ]:
outputdf = pandas.DataFrame(result, columns=header)
outputdf.to_csv('anime_matrix_true.csv', index=False)

In [5]:
df_temp = pandas.read_csv('anime_matrix_true.csv')

In [ ]:
#spot checking for anime matrix true
# out = []
# for i in range(len(df_anime)):
#     out.append(df_matrix.at[i,'Manzerek'])

# print(out)
print(df_matrix.loc[4163]['anime'])
print(df_matrix.at[4163,'clouds88'])


In [ ]:
df_matrix.to_csv('anime_matrix_temp.csv', index=False)

In [ ]:
#read each user's watch list and creates a bag of words of all anime they watched
result = []
length = len(df_user_info)

# text = 34156
# print(text)
# print(create_soup(df_anime.loc[df_anime['Id'] == text.index[0]]))

for j in tqdm(range(length)):
    try:
        user = df_user_info.loc[j]['user']
        watched = (df_user_info.loc[j]['watched list']).split()
        bag = ''
        for k in range(len(watched)):
            if watched[k] not in test_anime:
                try:
                    temp = '\'' + watched[k] + '\''
                    index = df_anime.loc[df_anime['Id'] == temp].index[0]
                    bag += ' ' + create_soup_reduced(df_anime.loc[index])
                except:
                    pass
    except:
        pass
    result.append([user,bag])

In [ ]:
outputdf = pandas.DataFrame(result, columns=['user','bag of words'])
outputdf.to_csv('users_bag_of_words_reduced.csv', index=False)

In [ ]:
wa = df_user_info.loc[2]['watched list'].split()

bag = ''
for i in tqdm(range(len(wa))):
    if wa[i] not in test_anime:
        try:
            temp = '\'' + wa[i] + '\''
            index = df_anime.loc[df_anime['Id'] == temp].index[0]
            bag += ' ' + create_soup(df_anime.loc[index])
        except:
            pass



In [14]:
#X is the list of anime found in df_anime
#y is if the the user watched the anime or not

df_anime_x = pandas.read_csv('anime_train.csv')
df_anime_y = pandas.read_csv('anime_matrix_true.csv')


In [13]:
print(df_anime_y.columns)

Index(['anime', 'ilampan', 'Zephyyr', 'Alister-Dark', 'AMechaDegenarate',
       'Matthew_Myers', 'eezairi', 'Neji4460', 'Aguena_Jun', 'andrzejszczelba',
       ...
       'Jaxelar', 'BitSneakySvenpai', 'PeelyPeelyBoi', 'Tray1990', 'Kilax',
       'Darkidda', 'chidoriP', 'snepback', 'Prism8Azure', 'urban_orange'],
      dtype='object', length=9242)


In [54]:
#creates bag of words with selected features
def create_soup_selection(x,studio=False,source=False,genre=False,va=False,all=False):
    result = x['Anime Title']

    if x['Studio'] != 'n/a' and (studio or all):
        result += ' ' + x['Studio']
    if x['Source Material'] != 'n/a' and (source or all):
        result += ' ' + x['Source Material']
    if x['Genre'] != 'n/a' and (genre or all):
        result += ' ' + x['Genre']
    if x['VAs'] != 'n/a' and (va or all):
        result += ' ' + x['VAs']

    punc = '[],\'\"'
    
    for ele in result:
        if ele in punc:
            result = result.replace(ele, "")

    result
    return result

def Sort_Tuple(tup):
    # getting length of list of tuples
    lst = len(tup)
    for i in range(0, lst):
            
        for j in range(0, lst-i-1):
            if (tup[j][1] < tup[j + 1][1]):
                temp = tup[j]
                tup[j]= tup[j + 1]
                tup[j + 1]= temp
    return tup

In [15]:
df_anime_y.drop(df_anime_y.tail(876).index,inplace=True)

print(df_anime_x.shape)
print(df_anime_y.shape)

(3499, 8)
(3499, 9242)


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_anime_x, df_anime_y, test_size=0.2, random_state=42)

In [17]:
#create user list
users = []
for i in range(len(df_user_info)):
    users.append(df_user_info.loc[i]['user'])

#create watch list based on df_anime_y
result = []
for j in tqdm(range(len(users))):
    watched = ''
    for i in range(len(df_anime_y)):
        if (df_anime_y.at[i,users[j]]):
            watched += str(df_anime_y.loc[i]['Id']) + ' '
    result.append([users[j],watched])

100%|██████████| 9241/9241 [02:41<00:00, 57.39it/s]


In [19]:
print(y_test.head)

<bound method NDFrame.head of          Id  ilampan  Zephyyr  Alister-Dark  AMechaDegenarate  Matthew_Myers  \
1650  23799        0        0             0                 0              0   
2456   1810        0        0             0                 0              0   
2232    658        0        0             0                 0              0   
1945   5129        0        0             0                 0              0   
309   36522        0        0             0                 0              0   
...     ...      ...      ...           ...               ...            ...   
3127  10338        0        0             0                 0              0   
744   40368        0        0             0                 0              0   
631   38804        0        0             0                 0              0   
1557  20085        0        0             0                 0              0   
2213    612        0        0             0                 0              0   

      eez

In [10]:
df_mini_watch_list = pandas.DataFrame(result, columns=['user', 'watch list'])

test_anime = []
for i in range(len(X_test)):
    test_anime.append(X_test.iloc[i]['Id'])

NameError: name 'result' is not defined

In [87]:
#read each user's watch list and creates a bag of words of all anime they watched
user_bag_result = []
length = len(users)

for j in tqdm(range(length)):
    user = df_mini_watch_list.loc[j]['user']
    watched = (df_mini_watch_list.loc[j]['watch list']).split()
    bag = ''
    for k in range(len(watched)):
        if watched[k] not in test_anime:
            try:
                temp = '\'' + watched[k] + '\''
                index = df_anime.loc[df_anime['Id'] == temp].index[0]
                bag += ' ' + create_soup_selection(df_anime.loc[index],genre=True,studio=True)
            except:
                pass
    user_bag_result.append([user,bag])

#creates a bag of words for each anime
anime_bag_result = []
for i in tqdm(range(len(df_anime_x))):
    anime = df_anime_x.iloc[i]['Id'].replace('\'','')
    bag = create_soup_selection(df_anime_x.iloc[i],genre=True,studio=True)
    anime_bag_result.append([anime,bag])

#output the user bag of words to csv
outputdf = pandas.DataFrame(user_bag_result, columns=['user','bag of words'])
outputdf.to_csv('users_bag_of_words_genre_studio.csv', index=False)

#output the anime bag of words to csv
outputdf = pandas.DataFrame(anime_bag_result, columns=['Id','bag of words'])
outputdf.to_csv('anime_bag_of_words_genre_studio.csv', index=False)

100%|██████████| 3499/3499 [00:00<00:00, 6127.97it/s]


In [24]:
df_anime_x_temp = df_anime_x.copy()

# df_anime_x_temp = df_anime_y[['ilampan']].copy()
user_column = df_anime_y['ilampan']
df_anime_x_temp['ilampan'] = user_column

In [122]:
columns = df_anime_y.columns
users = []
for col in columns[1:]:
    users.append(col)

In [23]:
user_column = df_anime_y[users[0]]
df_anime_x_temp[users[0]] = user_column

NameError: name 'df_anime_x_temp' is not defined

In [ ]:
#creates a bag of words for each anime
anime_bag_result = []
for i in tqdm(range(len(df_anime_x))):
    anime = df_anime_x.iloc[i]['Id'].replace('\'','')
    bag = create_soup_selection(df_anime_x.iloc[i],genre=True,studio=True)
    anime_bag_result.append([anime,bag])

In [25]:
print(users[0])

ilampan


In [38]:
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection / union)

In [31]:
df_anime_x_temp = df_anime_x.copy()

# df_anime_x_temp = df_anime_y[['ilampan']].copy()
user_column = df_anime_y[users[0]]
df_anime_x_temp[users[0]] = user_column

print(X_test.iloc[1])


Id                                                            '1810'
Anime Title                                 'KyoushiroutoTowanoSora'
Rating                                                          6.42
Studio                                                           TNK
Source Material                                                Manga
Demographic                                                'Shounen'
Genre                                ['Drama', 'Romance', 'Shounen']
VAs                ['MatsuokaYuki', 'YukiKonishi', 'KonishiKatsuy...
Name: 2456, dtype: object


In [102]:
all_distances = []
for j in tqdm(range(len(X_test))):
    bag1 = create_soup_selection(X_test.iloc[j],all=True).split()
    distances = []
    for i in range(len(X_train)):
        bag2 = create_soup_selection(X_train.iloc[i],all=True).split()
        distances.append((X_train.iloc[i]['Id'].replace('\'',''),jaccard(list(bag1),list(bag2))))
    Sort_Tuple(distances)
    all_distances.append(distances)


100%|██████████| 700/700 [10:16<00:00,  1.14it/s]


In [103]:
import pickle

# open a file, where you ant to store the data
file = open('knn_distances', 'wb')

# dump information to that file
pickle.dump(all_distances, file)

# close the file
file.close()

In [ ]:
import pickle

# open a file, where you stored the pickled data
file = open('knn_distances', 'rb')

# dump information to that file
all_distances = pickle.load(file)

# close the file
file.close()

In [100]:
print(distances[0:5])
print(len(distances))
print(distances[0][0])

[('143', 0.17777777777777778), ('10030', 0.13043478260869565), ('7674', 0.13043478260869565), ('12365', 0.13043478260869565), ('1128', 0.1111111111111111)]
2799
143


In [98]:
Sort_Tuple(distances)

print(distances[0:5])

print(df_anime_y.loc[df_anime_y['Id'] == 32937]['ilampan'])

# temp = df_anime_y.loc[df_anime_y['Id'] == 32937]['ilampan']
# print(df_anime_y.loc[df_anime_y['Id'] == 32937])
print(df_anime_y.head)
sum = 0
for i in range(5):
    sum += df_anime_y.loc[df_anime_y['Id'] == 32937]['Zephyyr']

print('sum = ',sum/2)




[('143', 0.17777777777777778), ('10030', 0.13043478260869565), ('7674', 0.13043478260869565), ('12365', 0.13043478260869565), ('1128', 0.1111111111111111)]
0    0
Name: ilampan, dtype: int64
<bound method NDFrame.head of          Id  ilampan  Zephyyr  Alister-Dark  AMechaDegenarate  Matthew_Myers  \
0     32937        0        1             0                 0              0   
1     33206        0        1             0                 0              0   
2     32615        0        1             0                 1              0   
3     33487        0        1             0                 0              0   
4     33506        0        1             0                 0              0   
...     ...      ...      ...           ...               ...            ...   
3494  16241        0        1             0                 0              0   
3495  17681        1        0             0                 0              0   
3496  17249        0        0             0                